## apple app store crawling




### 방법1 👉 encoding 오류 
  * https://chaelist.github.io/docs/webscraping/app_review/#app-store-scraping
  * https://signing.tistory.com/51

In [ ]:
pip install app_store_scraper

In [ ]:
from app_store_scraper import Podcast

import pandas as pd

import numpy as np

import json

ssc = AppStore(country='kr', app_id='1318541802', app_name='서울시민카드')

ssc.review() 

#📌 latin encoding 에서 error 발생 (왓챠 어플은 잘 되던데,,)

2022-06-03 12:46:13,376 [INFO] Base - Initialised: AppStore('kr', '서울시민카드', 1318541802)
2022-06-03 12:46:13,378 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/서울시민카드/id1318541802
2022-06-03 12:46:13,483 [ERROR] Base - Something went wrong: 'latin-1' codec can't encode characters in position 30-35: ordinal not in range(256)
2022-06-03 12:46:13,487 [INFO] Base - [id:1318541802] Fetched 0 reviews (0 fetched in total)


In [ ]:
print(ssc)

     Country | kr
        Name | 서울시민카드
          ID | 1318541802
         URL | https://apps.apple.com/kr/app/서울시민카드/id1318541802
Review count | 0


In [ ]:
ssc.reviews

[]

In [ ]:
 fetched_reviews = ssc.reviews
 ios_review_df = pd.DataFrame(fetched_reviews)

In [ ]:
ios_review_df

""


### 방법2

* https://itunes.apple.com/kr/rss/customerreviews/id=1318541802/sortBy=mostRecent/xml
  * 95개 리뷰 👉 50개의 사용자 리뷰 + 45개의 개발자 답글

In [ ]:
pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import xmltodict
from urllib.request import urlopen
import json


In [ ]:
url = "https://itunes.apple.com/kr/rss/customerreviews/id=1318541802/sortBy=mostRecent/xml"
response = urlopen(url).read()
response

b'<?xml version="1.0" encoding="utf-8"?>\n\n\t<feed xmlns:im="http://itunes.apple.com/rss" xmlns="http://www.w3.org/2005/Atom" xml:lang="ko">\n\t\t<id>https://mzstoreservices-int-st.itunes.apple.com/kr/rss/customerreviews/id=1318541802/sortBy=mostRecent/xml</id><title>iTunes Store:\xec\x82\xac\xec\x9a\xa9\xec\x9e\x90 \xed\x9b\x84\xea\xb8\xb0</title><updated>2022-06-03T05:53:38-07:00</updated><link rel="alternate" type="text/html" href="https://apps.apple.com/WebObjects/MZStore.woa/wa/viewGrouping?cc=kr&amp;id=29141"/><link rel="self" href="https://mzstoreservices-int-st.itunes.apple.com/kr/rss/customerreviews/id=1318541802/sortBy=mostRecent/xml"/><link rel="first" href="https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1318541802/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1318541802/sortBy=mostRecent/xml"/><link rel="last" href="https://itunes.apple.com/kr/rss/customerreviews/page=2/id=1318541802/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1318541802/sortBy=mostRec

In [ ]:
xml = xmltodict.parse(response)
XmlToJson = json.loads(json.dumps(xml))


In [ ]:
res_dict = []

for i in range(len(XmlToJson['feed']['entry'])):
    res_dict.append({
        'DATE' : XmlToJson['feed']['entry'][i]['updated'],
        'STAR' : XmlToJson['feed']['entry'][i]['im:rating'],
        'LIKE' : XmlToJson['feed']['entry'][i]['im:voteSum'],
        'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
        'TITLE' : XmlToJson['feed']['entry'][i]['title'],
        'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
    })


In [ ]:
len(res_dict)

50

In [ ]:
res_dict

[{'DATE': '2022-05-07T21:07:58-07:00',
  'DISLIKE': 0,
  'LIKE': '0',
  'REVIEW': '도서관 다닐 때 카드 안들고 다녀서 좋아요 위젯 생기면 더 편해질 것 같아요',
  'STAR': '5',
  'TITLE': 'ios위젯 만들어주셨으면 좋겠어요'},
 {'DATE': '2022-04-04T20:37:15-07:00',
  'DISLIKE': 0,
  'LIKE': '0',
  'REVIEW': '소소한 할인쿠폰도 많이 주는것같고 \n서울시 공공시설 이용할때 하나로 쓸수 있어서 좋은 것 같아요',
  'STAR': '5',
  'TITLE': '사용하기 편리해요'},
 {'DATE': '2022-03-29T18:16:54-07:00',
  'DISLIKE': 0,
  'LIKE': '0',
  'REVIEW': '아래에 저 같은 분이 계시던데 저두 문자로 인증하려구 하니 쿠키 제한에 걸려 본인 문자 인증이 안됩니다.\n\n당장 앱카드를 쓸수가 없어요 얼른 개선 부탁드립니다.\n\n아이폰 유저입니다',
  'STAR': '1',
  'TITLE': '쿠키 제한이 걸려서 문자 인증이안되요'},
 {'DATE': '2022-03-12T20:00:25-07:00',
  'DISLIKE': 0,
  'LIKE': '0',
  'REVIEW': '통신사를 선택하면  PASS선탹하는 버튼이 사라지고 문자인증만 남고, 인증문자도 한번에 안와서 겨우 받으면 세션만료 쿠키제한되어있다며 본인인증이 안됩니다.',
  'STAR': '1',
  'TITLE': '아이폰13인데요'},
 {'DATE': '2022-02-25T03:16:43-07:00',
  'DISLIKE': 0,
  'LIKE': '0',
  'REVIEW': 'Ios 업데이트 하면서 또 안되네요 \n개발자 누굽니까',
  'STAR': '1',
  'TITLE': '1점도 아깝다'},
 {'DATE': '2022-02-18T01:53:28-07:00'

In [ ]:
import pandas as pd

data = pd.DataFrame(data=[], columns=['날짜','싫어요','좋아요','리뷰','별점','제목'])

for i in range(len(res_dict)):
        tmp = []
        tmp.append(res_dict[i]['DATE'])
        tmp.append(res_dict[i]['DISLIKE'])
        tmp.append(res_dict[i]['LIKE'])
        tmp.append(res_dict[i]['REVIEW'])
        tmp.append(res_dict[i]['STAR'])
        tmp.append(res_dict[i]['TITLE'])
        
        # 수집한 1명의 리뷰를 결과 프레임에 추가합니다.
        tmp = pd.DataFrame(data=[tmp], columns=data.columns)
        data = pd.concat([data,tmp])

In [ ]:
data.shape

(50, 6)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.drop(['index'], axis=1,inplace=True)

In [ ]:
from google.colab import files

data.to_csv('ios_review.csv', encoding = 'utf-8-sig') 
files.download('ios_review.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## google app store crawling

* https://strangefate.github.io/2022/01/06/AppReview-googleplayscraper/
* https://github.com/JoMingyu/google-play-scraper 

* 웹페이지에 222개 리뷰 👉 119개의 사용자 리뷰 + 서울 시민카드 관리자 답글 103 개

In [ ]:
pip install google-play-scraper

In [ ]:
from google_play_scraper import app

result = app(
    'com.seoul.scc',
    lang='ko', # defaults to 'en'
    country='us' # defaults to 'us'
)

In [ ]:
from google_play_scraper import Sort, reviews_all

result = reviews_all(
    'com.seoul.scc',
    sleep_milliseconds=1000, # defaults to 0
    lang='ko', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    filter_score_with=None # defaults to None(means all score)
)

In [ ]:
result[0]

{'at': datetime.datetime(2018, 12, 5, 23, 28, 46),
 'content': '앱을 실행하고 관심분야를 선택하면 도서관 같은경우 구에 속한 도서관의 경우 카드가 한장이면 통합되있어서 다른곳도 이용가능한데 여기는 도서관마다 등록하라고 카드가 다 띄워지는데 한곳만 등록하면 통합이 되는건지 다른곳을 이용하려면 다중에카드를 등록해야하는지 안내가 되야하고 만약 통합이 안되면 통합이 되도록 시스템 변경이 있어야 할것 같습니다.',
 'repliedAt': datetime.datetime(2019, 1, 29, 6, 51, 2),
 'replyContent': '안녕하세요. 서울시민카드입니다.\n\n통합 회원으로 서비스하는 도서관이 늘고 있어 말씀 주신 것처럼 구립 도서관으로 카드를 등록하고 사용하실 수 있도록 준비 중입니다.\n\n지속적인 관심 부탁드립니다.\n\n고맙습니다.',
 'reviewCreatedVersion': '1.0.1',
 'reviewId': 'gp:AOqpTOEGOemcO2IPFrKKxBrom6giGa4iientOVIuYU5oPTF0_nSACobGsmRYEB8BjIvi8CkNghOPnhjfgkQ5uiE',
 'score': 3,
 'thumbsUpCount': 20,
 'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g',
 'userName': 'Google 사용자'}

In [ ]:
import pandas as pd

data = pd.DataFrame(data=[], columns=['날짜','리뷰','응답날짜','응답','별점','유저명','공감수'])

for i in range(len(result)):
        tmp = []
        tmp.append(result[i]['at'])
        tmp.append(result[i]['content'])
        tmp.append(result[i]['repliedAt'])
        tmp.append(result[i]['replyContent'])
        tmp.append(result[i]['score'])
        tmp.append(result[i]['userName'])
        tmp.append(result[i]['thumbsUpCount'])
        
        # 수집한 1명의 리뷰를 결과 프레임에 추가합니다.
        tmp = pd.DataFrame(data=[tmp], columns=data.columns)
        data = pd.concat([data,tmp])
        

In [ ]:
data.shape 

(119, 7)

In [ ]:
data.head(3)

,날짜,리뷰,응답날짜,응답,별점,유저명,공감수
0,2018-12-05 23:28:46,앱을 실행하고 관심분야를 선택하면 도서관 같은경우 구에 속한 도서관의 경우 카드가 ...,2019-01-29 06:51:02,안녕하세요. 서울시민카드입니다.\n\n통합 회원으로 서비스하는 도서관이 늘고 있어 ...,3,Google 사용자,20
0,2020-11-25 04:23:16,이런 시도 넘 좋아요. 매번 쓰는 게 아닌데 실물 카드 들고 다니는 게 진짜 귀찮았...,2020-12-11 05:08:21,안녕하세요. 서울시민카드 입니다. 리뷰 감사드립니다.,5,Kuei-fei Yang,38
0,2022-05-13 02:06:10,반납 연장을 어플 내에서 할 수 있으면 좋겠어요. 힘들면 그냥 사이트에서 연장한 거...,2022-05-16 08:49:33,"안녕하세요. 서울시민카드입니다.\n서울시민카드 앱 자체에서 대출연장 기능은 없으나,...",4,신은진,0


In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.drop(['index'], axis=1,inplace=True)

In [ ]:
data.head(2)

,날짜,리뷰,응답날짜,응답,별점,유저명,공감수
0,2018-12-05 23:28:46,앱을 실행하고 관심분야를 선택하면 도서관 같은경우 구에 속한 도서관의 경우 카드가 ...,2019-01-29 06:51:02,안녕하세요. 서울시민카드입니다.\n\n통합 회원으로 서비스하는 도서관이 늘고 있어 ...,3,Google 사용자,20
1,2020-11-25 04:23:16,이런 시도 넘 좋아요. 매번 쓰는 게 아닌데 실물 카드 들고 다니는 게 진짜 귀찮았...,2020-12-11 05:08:21,안녕하세요. 서울시민카드 입니다. 리뷰 감사드립니다.,5,Kuei-fei Yang,38


In [ ]:
from google.colab import files

data.to_csv('review.csv', encoding = 'utf-8-sig') 
files.download('review.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>